In [2]:
from collections import defaultdict
import numpy as np
import sqlite3 as s

In [25]:
#数据库操作类
class s3db:
    """sqlite3数据库功能封装"""

    def __init__(self, dbpath=None):
        """如果构造时给出了数据库路径,则直接打开"""
        self.conn = None
        if dbpath is not None:
            self.open(dbpath)

    def opened(self):
        """判断是否打开了数据库"""
        return self.conn is not None

    def open(self, dbpath):
        """打开指定的数据库,返回值:是否成功"""
        if self.conn is not None:
            return True

        try:
            self.conn = s.connect(dbpath, check_same_thread=False)
            return True
        except Exception as e:
            return False

    def opt_set(self, cmd, val):
        """设定sqlite3配置参数,返回值:是否成功"""
        if self.conn is None:
            return False

        try:
            if not isinstance(val, str):
                val = str(val)
            self.conn.execute("PRAGMA  %s = %s" % (cmd, val))
            return True
        except Exception as e:
            return False

    def opt_def(self):
        """设置默认优化参数"""
        self.opt_set('Synchronous', 'OFF')
        self.opt_set('Journal_Mode', 'WAL')
        self.opt_set('Cache_Size', '16384')

    def close(self):
        """关闭数据库连接"""
        if self.conn is None:
            return True
        self.conn.close()
        self.conn = None

    def exec(self, sql, data=None, commit=True):
        """执行sql语句(不返回结果集的增删改语句).
            data - sql的绑定参数
            commit - 告知是否立即提交.不是立即提交时需要外部执行commit()方法手动提交,提高性能.
            返回值: (是否成功,错误消息)
        """
        try:
            if data is None:
                self.conn.execute(sql)
            else:
                self.conn.execute(sql, data)
            if commit:
                self.conn.commit()
            return True, ''
        except Exception as e:
            self.conn.rollback()
            return False, str(e)

    def commit(self):
        """手动提交执行结果.返回值(是否成功,错误消息)"""
        try:
            self.conn.commit()
            return True, ''
        except Exception as e:
            self.conn.rollback()
            return False, str(e)

class s3query:
    """sqlite3数据查询功能封装"""

    def __init__(self, db):
        """构造时绑定数据库对象"""
        self.db = db
        if db.conn:
            self.open(db)

    # 初始化查询对象,可指定数据库对象
    def open(self, db=None):
        """创建查询游标对象."""
        if db is None:
            db = self.db
        self.conn = db.conn
        self.cur = db.conn.cursor()

    def exec(self, sql, param=None, cmt=True):
        """执行sql语句,不要求获取结果集.
            返回值:成功时 - (True,影响的行数)
                  失败时 - (False,错误信息)
        """
        try:
            if param is None:
                self.cur.execute(sql)
            else:
                self.cur.execute(sql, param)
            if cmt:
                self.conn.commit()
            return True, self.cur.rowcount
        except Exception as e:
            self.conn.rollback()
            return False, str(e)

    def query(self, sql, param=None, fetchsize=None):
        """执行sql查询,得到结果集(默认是得到全部,也可以指定获取的数量)
            返回值:成功时 - (结果集,'')
                  失败时 - (None,错误信息)
        """
        try:
            if param is None:
                self.cur.execute(sql)
            else:
                self.cur.execute(sql, param)
            if fetchsize is None:
                return self.cur.fetchall(), ''
            else:
                return self.cur.fetchmany(size=fetchsize), ''
        except Exception as e:
            return None, str(e)

    def fetch(self, fetchsize):
        """在query使用了fetchsize分批获取之后,继续获取结果集的后续部分
            返回值:成功时 - (结果集,'')
                  失败时 - (None,错误信息)
        """
        try:
            return self.cur.fetchmany(size=fetchsize), ''
        except Exception as e:
            return None, str(e)

    def append(self, obj, cmt=True):
        """轻量级ORM插入实现,obj的类型为表名,obj内含属性为表中字段与对应的值
            返回值:成功时 - (True,影响的行数)
                  失败时 - (False,错误信息)
        """

        def _insert(obj, cmt):
            tbl = type(obj).__name__
            val = []
            fds = []
            dmy = []
            for f in obj.__dict__:
                fds.append(f)
                val.append(obj.__dict__[f])
                dmy.append('?')
            sql = 'insert into %s (%s) values(%s)' % (tbl, ','.join(fds), ','.join(dmy))
            return self.exec(sql, val, cmt)

        if isinstance(obj, list) or isinstance(obj, tuple):
            for o in obj:
                _insert(o, False)
            if cmt:
                return self.db.commit()
            else:
                return True, ''
        else:
            return _insert(obj, cmt)

    def extract(self, sql, filter_fun, param=None, fetchsize=100):
        """执行查询,给出sql和参数param,对结果行进行filter_fun过滤,可设定提取批尺寸fetchsize.
           返回值:(结果数量,错误信息)
        """
        rc = 0
        rows, msg = self.query(sql, param, fetchsize)
        if msg: return rc, msg
        while len(rows):
            for row in rows:
                filter_fun(row)
                rc += 1
            rows, msg = self.fetch(fetchsize)
            if msg: return rc, msg
        return rc, msg

    def has(self, name, type='table'):
        """判断指定的库表对象table/index/view是否存在.
            返回值:None - 查询失败,结果未知
                  True/False - 告知是否存在
        """
        rows, msg = self.query("SELECT name FROM sqlite_master WHERE type=? and name=?", (type, name))
        if msg != '':
            return None
        return len(rows) > 0

    def close(self):
        """关闭数据查询对象"""
        if self.cur is not None:
            self.cur.close()
        self.cur = None
        self.conn = None
        self.db = None


In [29]:
#读取文本

#连接数据库
db=s3db('E:/DATA/Corporation/ner_data/samples.sqlite3')
print('connection:',db.opened())

#数据库sql操作
s3q=s3query(db)
sql='select * from tbl_datas'

t=s3q.query(sql,fetchsize=1)
print(t)
print("\n\n\n")
q=s3q.fetch(1)
print(q)

print("\n\n\n")
q=s3q.fetch(1)
print(q)

connection: True
([('63ff4afb18c6c7aded542b08da1998b0', '龙川县博物馆馆藏文物预防性保护项目中标(成交)结果公告', '\n一、采购计划编号:441622-2022-00205\n二、项目编号:GDDRJC202201008\n三、项目名称:龙川县博物馆馆藏文物预防性保护项目\n四、采购结果\n合同包1(龙川县博物馆馆藏文物预防性保护项目):\n供应商名称\n供应商地址\n中标(成交)金额\n深圳市华图测控系统有限公司\n深圳市宝安区西乡街道铁岗社区铁岗水库路147号厂房2栋3层\n1,679,000.00元\n五、主要标的信息\n合同包1(龙川县博物馆馆藏文物预防性保护项目):\n货物类(深圳市华图测控系统有限公司)\n品目号\n品目名称\n采购标的\n品牌\n规格型号\n数量(单位)\n单价(元)\n总价(元)\n1-1\n其他陈列品\n龙川县博物馆藏文物预防性保护项目\n详见附件\n详见附件\n1(项)\n1,679,000.00\n1,679,000.00\n六、评审专家(单一来源采购人员)名单:\n评审委员会总人数:3\n随机抽取专家名单:林仕安 丘志坚\n采购人代表名单:黄晓瑞\n自行选定专家名单:/\n七、代理服务收费标准及金额:\n代理服务费收费标准:\n按国家发改委(发改价格[2011]534号)文规定进行收取。\n代理服务费金额:\n合同包1(龙川县博物馆馆藏文物预防性保护项目):2.2469万元。\n收取对象:中标(成交)供应商。\n八、公告期限\n自本公告发布之日起1个工作日。\n九、其他补充事宜\n合同包1(龙川县博物馆馆藏文物预防性保护项目):\n供应商\n资格性审查\n符合性审查\n技术得分\n商务得分\n价格得分\n综合得分\n得分排名\n推荐排名\n深圳市华图测控系统有限公司\n通过\n通过\n47.33\n19.33\n29.74\n96.40\n1\n1\n安徽宝立文物保护科技发展有限公司\n通过\n通过\n32.00\n7.33\n29.72\n69.05\n2\n2\n深圳市百德盛电子有限公司\n通过\n通过\n28.67\n3.33\n30.00\n62.00\n3\n3\n十、凡对本次公告内容提出询问,请按以下方式联系。\n1.釆购人信息\n名 称:龙川县博物馆\n

## 统计词频

In [3]:
n = 4
min_count = 0
ngrams = defaultdict(int)

texts=['四是四十一是十十四是十四四十一是四十一']


for t in texts:
    for i in range(len(t)):
        for j in range(1, n+1):
            if i+j <= len(t):
                ngrams[t[i:i+j]] += 1
#保留词频数>=min_count的词 (n<=4)
ngrams = {i:j for i,j in ngrams.items() if j >= min_count}
#统计了的词的总数 统计两个字以上的词的个数
total = 1.*sum([j for i,j in ngrams.items() if len(i) == 1])
print('ngrams:',ngrams)
print(total)

ngrams: {'四': 6, '四是': 2, '四是四': 1, '四是四十': 1, '是': 4, '是四': 2, '是四十': 2, '是四十一': 2, '四十': 3, '四十一': 3, '四十一是': 2, '十': 6, '十一': 3, '十一是': 2, '十一是十': 1, '一': 3, '一是': 2, '一是十': 1, '一是十十': 1, '是十': 2, '是十十': 1, '是十十四': 1, '十十': 1, '十十四': 1, '十十四是': 1, '十四': 2, '十四是': 1, '十四是十': 1, '四是十': 1, '四是十四': 1, '是十四': 1, '是十四四': 1, '十四四': 1, '十四四十': 1, '四四': 1, '四四十': 1, '四四十一': 1, '十一是四': 1, '一是四': 1, '一是四十': 1}
19.0


## 凝固度筛选

In [4]:
min_proba = {2:1.2, 3:20, 4:30}

def is_keep(s, min_proba):
    if len(s) >= 2:
        score=2**31
        for i in range(len(s)-1):
            print(s,s[:i+1],s[i+1:])
            print(ngrams[s],ngrams[s[:i+1]],ngrams[s[i+1:]])
            score=min(score,total*ngrams[s]/(ngrams[s[:i+1]]*ngrams[s[i+1:]]))
        if score > min_proba[len(s)]:
            return True
    else:
        return False
#凝固度满足的词(长度>=2)
ngrams_ = set(i for i,j in ngrams.items() if is_keep(i, min_proba))
print('ngrams_:',ngrams_)

四是 四 是
2 6 4
四是四 四 是四
1 6 2
四是四 四是 四
1 2 6
四是四十 四 是四十
1 6 2
四是四十 四是 四十
1 2 3
四是四十 四是四 十
1 1 6
是四 是 四
2 4 6
是四十 是 四十
2 4 3
是四十 是四 十
2 2 6
是四十一 是 四十一
2 4 3
是四十一 是四 十一
2 2 3
是四十一 是四十 一
2 2 3
四十 四 十
3 6 6
四十一 四 十一
3 6 3
四十一 四十 一
3 3 3
四十一是 四 十一是
2 6 2
四十一是 四十 一是
2 3 2
四十一是 四十一 是
2 3 4
十一 十 一
3 6 3
十一是 十 一是
2 6 2
十一是 十一 是
2 3 4
十一是十 十 一是十
1 6 1
十一是十 十一 是十
1 3 2
十一是十 十一是 十
1 2 6
一是 一 是
2 3 4
一是十 一 是十
1 3 2
一是十 一是 十
1 2 6
一是十十 一 是十十
1 3 1
一是十十 一是 十十
1 2 1
一是十十 一是十 十
1 1 6
是十 是 十
2 4 6
是十十 是 十十
1 4 1
是十十 是十 十
1 2 6
是十十四 是 十十四
1 4 1
是十十四 是十 十四
1 2 2
是十十四 是十十 四
1 1 6
十十 十 十
1 6 6
十十四 十 十四
1 6 2
十十四 十十 四
1 1 6
十十四是 十 十四是
1 6 1
十十四是 十十 四是
1 1 2
十十四是 十十四 是
1 1 4
十四 十 四
2 6 6
十四是 十 四是
1 6 2
十四是 十四 是
1 2 4
十四是十 十 四是十
1 6 1
十四是十 十四 是十
1 2 2
十四是十 十四是 十
1 1 6
四是十 四 是十
1 6 2
四是十 四是 十
1 2 6
四是十四 四 是十四
1 6 1
四是十四 四是 十四
1 2 2
四是十四 四是十 四
1 1 6
是十四 是 十四
1 4 2
是十四 是十 四
1 2 6
是十四四 是 十四四
1 4 1
是十四四 是十 四四
1 2 1
是十四四 是十四 四
1 1 6
十四四 十 四四
1 6 1
十四四 十四 四
1 2 6
十四四十 十 四四十
1 6 1
十四四十 十四 四十
1 2 3
十四四十 十四四 十
1 1 6
四四 四 

## 切分

In [18]:
def cut(s):
    r = np.array([0]*(len(s)-1))
    for i in range(len(s)-1):
        for j in range(2, n+1):
            if i+j>len(s):
                break
            if s[i:i+j] in ngrams_:
                r[i:i+j-1] += 1
    w = [s[0]]
    for i in range(1, len(s)):
        if r[i-1] > 0:
            w[-1] += s[i]
        else:
            w.append(s[i])
    return w

words = defaultdict(int)
for t in texts:
    for i in cut(t):
        words[i] += 1

words = {i:j for i,j in words.items() if j >= min_count}
print(words)

{'四': 3, '是': 4, '四十一': 3, '十': 3}


## 回溯

In [22]:
# def is_real(s):
#     if len(s) >= 3:
#         for i in range(3, n+1):
#             for j in range(len(s)-i+1):
#                 if s[j:j+i] not in ngrams_:
#                     return False
#         return True
#     else:
#         return True

def is_real(s):
    if len(s)==1:
        return True
    if len(s)<=n:
        if s in ngrams_:
            return True
        for i in range(2,len(s)):
            flag=True
            for j in range(len(s)-i+1):
                if s[j:j+i] not in ngrams_:
                    flag=False
                    continue
            if flag:
                return True
        return False
    else:
        for i in range(2,n+1):
            flag=True
            for j in range(len(s)-i+1):
                if s[j:j+i] not in ngrams_:
                    flag=False
                    continue
            if flag:
                return True
        return False







w = {i:j for i,j in words.items() if is_real(i)}
print(w)

{'四': 3, '是': 4, '四十一': 3, '十': 3}
